In [26]:
import numpy as np
import cv2

#tutaj src to obraz, z ktorego piksele beda wklejane do obrazu dst
#feather amount to procent, sluzacy do sterowania wielkoscia obszaru, ktory bedzie poddany wagowaniu
def blendImages(src, dst, mask, featherAmount=0.2):
    #indeksy nie czarnych pikseli maski
    maskIndices = np.where(mask != 0)
    #te same indeksy tylko, ze teraz w jednej macierzy, gdzie kazdy wiersz to jeden piksel (x, y)
    maskPts = np.hstack((maskIndices[1][:, np.newaxis], maskIndices[0][:, np.newaxis]))
    faceSize = np.max(maskPts, axis=0) - np.min(maskPts, axis=0)
    featherAmount = featherAmount * np.max(faceSize)

    hull = cv2.convexHull(maskPts)
    dists = np.zeros(maskPts.shape[0])
    for i in range(maskPts.shape[0]):
        dists[i] = cv2.pointPolygonTest(hull, (maskPts[i, 0], maskPts[i, 1]), True)

    weights = np.clip(dists / featherAmount, 0, 1)

    composedImg = np.copy(dst)
    composedImg[maskIndices[0], maskIndices[1]] = weights[:, np.newaxis] * src[maskIndices[0], maskIndices[1]] + (1 - weights[:, np.newaxis]) * dst[maskIndices[0], maskIndices[1]]

    return composedImg

#uwaga, tutaj src to obraz, z ktorego brany bedzie kolor
def colorTransfer(src, dst, mask):
    transferredDst = np.copy(dst)
    #indeksy nie czarnych pikseli maski
    maskIndices = np.where(mask != 0)
    #src[maskIndices[0], maskIndices[1]] zwraca piksele w nie czarnym obszarze maski

    maskedSrc = src[maskIndices[0], maskIndices[1]].astype(np.int32)
    maskedDst = dst[maskIndices[0], maskIndices[1]].astype(np.int32)
    print(maskedDst.shape)
    meanSrc = np.mean(maskedSrc, axis=0)
    meanDst = np.mean(maskedDst, axis=0)
    print(maskedDst)
    print(meanDst)
    print(meanSrc)
    print("hi")
    maskedDst = maskedDst - meanDst
    maskedDst = maskedDst + meanSrc
    print(maskedDst.shape)
    maskedDst = np.clip(maskedDst, 0, 255)
    print(maskedDst.shape)
    print("1")
    transferredDst[maskIndices[0], maskIndices[1]] = maskedDst
    print(transferredDst.shape)
    return transferredDst



In [27]:
import numpy as np
import cv2

import ImageProcessing

handImg = cv2.imread("../data/hand.png")
eyeImg = cv2.imread("../data/eye.png")
maskImg = cv2.imread("../data/mask.png")

#zmiana obrazka kolorowego na obrazek w skali szarosci
mask = np.mean(maskImg, axis=2)

eyeImg = colorTransfer(handImg, eyeImg, mask)
blendedImg = blendImages(eyeImg, handImg, mask)

cv2.imwrite("../eyeHandBlend.jpg", blendedImg)

(18857, 3)
[[151 172 217]
 [148 169 214]
 [152 175 220]
 ...
 [148 174 211]
 [150 176 212]
 [146 172 208]]
[132.15951636 149.8547489  191.09200827]
[143.86190804 165.88285517 223.60831521]
hi
(18857, 3)
(18857, 3)
1
(744, 558, 3)


True